In [ ]:
"""
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
"""

# Introduction

**Brief description of the problem and data**

On this excerces we have in our hands 300 of beautiful painting made by Monet.
This guy was a French painter and founder of Impressionism painting who is seen as a key precursor to modernism, that lived from 1840 to 1926.*

On this exercice the objective is to make a digital Monet, this fake monet is an AI that looks a a beutiful landscape from outsuide and paints as Monet. 

This task is going to need a GAN.

GAN are defined by amazon web services as a deep learning architecture where you train two neural networks to compete with each other to generate new, more authentic data from a given training data set.

In this case we have some images from Monet paintings. They are in .jpg and .tfrec formats. This .tfrec is basically "a custom TensorFlow format for storing a sequence of binary records. TFRecords are highly optimized for TensorFlow, which lead to ... efficient form of data storage and faster read speed compared to other types of formats".*

As in this case i do't intent to use TPU we will use images in .jpg format.

**Analysis**

This is not going to be an easy task, those are very preatty, and also preatty heterogeneus.

After researching a little bit, and wasting a hole previous notbook building another GAN that failed. I camed into a tutorial in keras that trains a CycleGAN for horses to look like zebras. Voila, lest go.

My hole notbook is just an adaptation of their tutorial, and hase more comments in it. The turorial is available in https://keras.io/examples/generative/cyclegan/

We are now going to:
1. Transform our data to look similar to data on the tutorial
2. Construct Cycle GAN

Most of the info is commented on code

**Cycle GAN\***

CycleGAN allows image-to-image translation without paired images. It learns the features of the target style and transforms images from the original style to match it. 

In fact we need a group of networks that generates images as follows:

Original Image (Monet painting) **Pases trough →** Generator G(trained digital Monet) **Creates→** Transformed Image (Fake Painting)

But also for training we need a generator that reverses the proces, so: Transformed Image (Fake Painting) → Generator F **(Creates →)** Reconstructed Image (Original Photo)

\* <br> https://keras.io/examples/keras_recipes/creating_tfrecords/ <br> https://en.wikipedia.org/wiki/Claude_Monet <br>https://www.geeksforgeeks.org/cycle-generative-adversarial-network-cyclegan-2/

# EDA

Lest go with data reading some basic exploration on paintings, RBG of the mean and dominant colors.

In [ ]:
import os
import keras
import glob
import cv2

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds

from PIL import Image
from sklearn.cluster import KMeans
from keras import layers, ops

In [ ]:
#RAW data
imgs = glob.glob("/kaggle/input/gan-getting-started/monet_jpg/*.jpg")
photos = glob.glob("/kaggle/input/gan-getting-started/photo_jpg/*.jpg")
N = min(10, len(imgs)) #Images to display on the output (EDA section)

In [ ]:
#NN
#Looking at some preatty paintings
print(f'Number of Monet images: {len(imgs)}')
img = Image.open(imgs[0])
print(f'Size is {img.size}')

fig, axes = plt.subplots(2, 5, figsize=(12, 6))

for ax, imgs in zip(axes.flatten(), imgs[:10]): #[x:y] desde x:y imágenes
    img = Image.open(imgs)
    ax.imshow(img)
    ax.axis("off")

plt.tight_layout()
plt.show()

In [ ]:
#NN
#Looking at some preatty Landscapes
print(f'Number of Landscapes: {len(photos)}')
print(f'Size is {img.size} px')

#IMAGEN
img = Image.open(photos[0])
fig, axes = plt.subplots(2, 5, figsize=(12, 6))

for ax, lands in zip(axes.flatten(), photos[:10]): #[x:y] desde x:y imágenes
    img = Image.open(lands)
    ax.imshow(img)
    ax.axis("off")

plt.tight_layout()
plt.show()

In [ ]:
#NN
#RGB hystograms of individual monet paintings
imgs = glob.glob("/kaggle/input/gan-getting-started/monet_jpg/*.jpg")

# Create subplots for displaying images and histograms
fig, axes = plt.subplots(2, 5, figsize=(12, 6))

# Iterate through images and axes
for idx, img_path in enumerate(imgs[:5]): # [:5] shoul match with subplot size (2, 5)
    # Open the image with PIL
    img = Image.open(img_path)
    
    # Show the image in the first column of the current row (left side)
    ax_img = axes[0, idx]  # First row for images
    ax_img.imshow(img)
    ax_img.axis("off")

    # Convert PIL image to NumPy array (RGB)
    img_rgb = np.array(img.convert('RGB'))
    # Convert to BGR for OpenCV processing (as OpenCV uses BGR)
    img_bgr = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2BGR)
    # Plot the RGB histogram in the second column of the current row (right side)
    ax_hist = axes[1, idx]  # Second row for histograms
    colors = ('r', 'g', 'b')
    for channel, col in enumerate(colors):
        hist = cv2.calcHist([img_bgr], [channel], None, [256], [0, 256])
        ax_hist.plot(hist, color=col, label=f'{col.upper()} channel')
    
    ax_hist.set_title("RGB Histogram")
    ax_hist.set_xlabel("Pixel Intensity")
    ax_hist.set_ylabel("Frequency")
    ax_hist.legend()

plt.tight_layout()
plt.show()


How pictures are a lot diferent between each other. This is going to be hard for our robotic Monet

In [ ]:
#NN
#Averaged RBG histogram on paintings
# Initialize accumulators for histograms per channel
hist_r = np.zeros((256, 1), dtype=np.float64)
hist_g = np.zeros((256, 1), dtype=np.float64)
hist_b = np.zeros((256, 1), dtype=np.float64)
total_pixels = 0

# Loop through all images in the dataset
for image_path in imgs:
    img = cv2.imread(image_path)
    if img is None:
        continue
    # Convert from BGR to RGB
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Update histograms for each channel
    hist_r += cv2.calcHist([img_rgb], [0], None, [256], [0, 256])
    hist_g += cv2.calcHist([img_rgb], [1], None, [256], [0, 256])
    hist_b += cv2.calcHist([img_rgb], [2], None, [256], [0, 256])
    
    # Count total pixels processed (images might have different sizes)
    total_pixels += img.shape[0] * img.shape[1]

# Plot aggregated histograms
plt.figure(figsize=(10, 5))
plt.plot(hist_r, color='r', label='Red')
plt.plot(hist_g, color='g', label='Green')
plt.plot(hist_b, color='b', label='Blue')
plt.title("Aggregated RGB Histogram (raw counts)")
plt.xlabel("Pixel Intensity")
plt.ylabel("Frequency")
plt.legend()
plt.show()

In [ ]:
#NN
# Bright or light on individual pintings
#Create subplots with N rows and 2 columns (one for image, one for its brightness histogram)
fig, axes = plt.subplots(nrows=N, ncols=2, figsize=(12, N * 3), squeeze=False)

for i, image_path in enumerate(imgs[:N]):
    # Read image using OpenCV
    img = cv2.imread(image_path)
    if img is None:
        continue
    # Convert image to RGB (for display) and to grayscale (for brightness analysis)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Display the original image in the left subplot
    axes[i, 0].imshow(img_rgb)
    axes[i, 0].set_title("Image")
    axes[i, 0].axis("off")
    
    # Compute the brightness histogram for the grayscale image
    hist = cv2.calcHist([gray], [0], None, [256], [0, 256])
    axes[i, 1].plot(hist, color='black')
    axes[i, 1].set_title("Brightness Histogram")
    axes[i, 1].set_xlabel("Pixel Intensity")
    axes[i, 1].set_ylabel("Frequency")

plt.tight_layout()
plt.show()

In [ ]:
#NN
#Bright of ligh on all paintings
#Initialize an accumulator for the brightness histogram and a counter for total pixels
hist_sum = np.zeros((256, 1), dtype=np.float64)
total_pixels = 0

# Loop over all images and accumulate the brightness histogram
for image_path in imgs:
    img = cv2.imread(image_path)
    if img is None:
        continue
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    hist = cv2.calcHist([gray], [0], None, [256], [0, 256])
    hist_sum += hist
    total_pixels += gray.shape[0] * gray.shape[1]

# Option 1: Plot the raw aggregated brightness histogram
plt.figure(figsize=(10, 5))
plt.plot(hist_sum, color='black')
plt.title("Aggregated Brightness Histogram (Raw Counts)")
plt.xlabel("Pixel Intensity")
plt.ylabel("Frequency")
plt.show()


# CycleGAN 1. Data transformation

It was fun to know color and light as a person who likes painting it was fun.
But lest go with our AI Monet.

In tutorial data is organized in a very specific way, as code is not that easy to follow, the less i change the less i break so transform data to be similar is key part.

In [ ]:
#Read data
# Get lists of file paths for each domain
monet_paths = glob.glob("/kaggle/input/gan-getting-started/monet_jpg/*.jpg")
photo_paths = glob.glob("/kaggle/input/gan-getting-started/photo_jpg/*.jpg")

# Create datasets from file paths
monet_ds = tf.data.Dataset.from_tensor_slices(monet_paths)
photo_ds = tf.data.Dataset.from_tensor_slices(photo_paths)

# Function to load an image, convert pixel values to [-1, 1], and assign a label.
def load_monet_image(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)  # Scales to [0, 1]
    image = (image - 0.5) * 2  # Now in [-1, 1]
    return image

def load_photo_image(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)  # Scales to [0, 1]
    image = (image - 0.5) * 2  # Now in [-1, 1]
    return image

# Map the functions to load and decode images with labels
monet_ds = (
    monet_ds.map(load_monet_image, num_parallel_calls=tf.data.AUTOTUNE)
    .shuffle(buffer_size=1000)
)

photo_ds = (
     photo_ds.map(load_photo_image, num_parallel_calls=tf.data.AUTOTUNE)
     .shuffle(buffer_size=1000)
 )

# Organize datasets in a dictionary similar to CycleGAN's "trainA" and "trainB"
dataset = {
    "trainA": monet_ds,
    "trainB": photo_ds
}


In [ ]:
#buffer and batches
buffer_size = 256
batch_size = 1

#loading datasets
train_monet, train_photos = dataset["trainA"], dataset["trainB"]
train_monet = (
    train_monet.map(lambda img: img, num_parallel_calls=tf.data.AUTOTUNE)
    .cache()
    .shuffle(buffer_size)
    .batch(batch_size)
)
train_photos = (
    train_photos.map(lambda img: img, num_parallel_calls=tf.data.AUTOTUNE)
    .cache()
    .shuffle(buffer_size)
    .batch(batch_size)
)

In [ ]:
#NN
# Extract one sample from each dataset (assuming each sample is a tuple: (image, label))
#Random because of .shuffle on dataset
imgA = next(iter(dataset["trainA"].take(1)))
imgB = next(iter(dataset["trainB"].take(1)))

# Create a figure with 2 rows and 1 column (2x1 subplot)
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(6, 12))

# Plot the first sample (Monet's Painting)
axes[0].imshow((imgA.numpy() + 1) / 2)  # Convert from [-1, 1] to [0, 1] for display
axes[0].set_title("Monet's Painting")
axes[0].axis("off")

# Plot the second sample (Real Landscape)
axes[1].imshow((imgB.numpy() + 1) / 2)  # Convert from [-1, 1] to [0, 1] for display
axes[1].set_title("Real Landscape")
axes[1].axis("off")

plt.tight_layout()
plt.show()


# Cycle GAN 2. Constructing NN

## Important variables:

**kernel_initializer (kernel_init):**

This initializer is used by layers like Conv2D or Conv2DTranspose to set the initial values for the convolutional filters. Random normal distribution with a mean of 0.0 and a standard deviation of 0.02. This choice is common in GAN architectures because initializing weights with small random values can help stabilize training.

**gamma_initializer (gamma_init):**
This is used by normalization layers (in this case, GroupNormalization) for the scaling parameter (often referred to as "gamma"). It controls how much the normalized output is scaled before being passed to the next layer. Using a similar random normal initializer (mean=0.0, stddev=0.02) is also a common practice in GANs to ensure that the initial scaling doesn't cause large deviations, helping to maintain stable gradients during training. Is a learned parameter.



In [ ]:
#Important Variables
autotune = tf.data.AUTOTUNE
#Kernel_init
kernel_init = keras.initializers.RandomNormal(mean=0.0, stddev=0.02)
# Gamma initializer for instance normalization.
gamma_init = keras.initializers.RandomNormal(mean=0.0, stddev=0.02)
#img size
input_img_size = (256, 256, 3)


## Basic Functions

In [ ]:
#Building blocks used in the CycleGAN generators and discriminators

"""
-------------------
Block of RNN
Recieves x, type of activation function, kernel init, size, stride, uses padding same, gamma init an bias as false
    
    1. Convolution + normalization + activation funtion
    2. Convolution + normalization
    3. Residual or skip conection from initial x to final x

dim = x.shape[-1]: Number of channels of RGB input - In a residual block, the goal is to learn a transformation that can be added back to the original input (the skip connection). For this addition to work, the input and the transformed output must have the same shape. By using x.shape[-1] (the number of channels in the input) as the number of filters for the convolutional layers, you ensure that the output of the block has the same number of channels as the input, making the element-wise addition possible. This design is a standard practice in architectures like ResNet and CycleGAN, where preserving the feature dimensions across a block helps in learning the residual mapping effectively.

"""

def residual_block(
    x,
    activation,
    kernel_initializer=kernel_init,
    kernel_size=(3, 3),
    strides=(1, 1),
    padding="same",  # Use 'same' padding instead of reflection padding
    gamma_initializer=gamma_init,
    use_bias=False,
    ):
    
    dim = x.shape[-1] 
    input_tensor = x

    # First convolutional layer with same padding
    x = layers.Conv2D(
        dim, # dim is the number of filters
        kernel_size,
        strides=strides,
        kernel_initializer=kernel_initializer,
        padding=padding,
        use_bias=use_bias,
    )(input_tensor)
    x = keras.layers.GroupNormalization(groups=1, gamma_initializer=gamma_initializer)(x)
    x = activation(x)

    # Second convolutional layer with same padding
    x = layers.Conv2D(
        dim,
        kernel_size,
        strides=strides,
        kernel_initializer=kernel_initializer,
        padding=padding,
        use_bias=use_bias,
    )(x)
    x = keras.layers.GroupNormalization(groups=1, gamma_initializer=gamma_initializer)(x)

    # Add the input tensor (residual connection)
    x = layers.add([input_tensor, x])
    return x
    
"""
-------------------
Downsample function 
Recieves x, type of activation function, kernel init, size, stride, uses padding same, gamma init an bias as false
    
    1. Convolution + normalization (with a bigger stride to downsample)
"""

def downsample(
    x,
    filters,
    activation,
    kernel_initializer=kernel_init,
    kernel_size=(3, 3),
    strides=(2, 2),
    padding="same",
    gamma_initializer=gamma_init,
    use_bias=False,
):
    x = layers.Conv2D(
        filters,
        kernel_size,
        strides=strides,
        kernel_initializer=kernel_initializer,
        padding=padding,
        use_bias=use_bias,
    )(x)
    x = keras.layers.GroupNormalization(groups=1, gamma_initializer=gamma_initializer)(
        x
    )
    if activation:
        x = activation(x)
    return x
    
"""
-------------------
Upsample function 
Recieves x, type of activation function, kernel init, size, stride, uses padding same, gamma init an bias as false
    
    1. Inverse - Convolution + normalization (with a bigger stride to downsample)
"""

def upsample(
    x,
    filters,
    activation,
    kernel_size=(3, 3),
    strides=(2, 2),
    padding="same",
    kernel_initializer=kernel_init,
    gamma_initializer=gamma_init,
    use_bias=False,
):
    x = layers.Conv2DTranspose(
        filters,
        kernel_size,
        strides=strides,
        padding=padding,
        kernel_initializer=kernel_initializer,
        use_bias=use_bias,
    )(x)
    x = keras.layers.GroupNormalization(groups=1, gamma_initializer=gamma_initializer)(
        x
    )
    if activation:
        x = activation(x)
    return x

## RESNet Generator (Function)

In [ ]:
"""
--------------
1. Convolution + Normalization + activation
2. Downsampling - uses funtion defined previously
3. Residual blocks - uses funtion defined previously
3. Upsampling - uses funtion defined previously
4. Final convolution and activtion

"""
def get_resnet_generator(
    filters=64,
    num_downsampling_blocks=2,
    num_residual_blocks=9,
    num_upsample_blocks=2,
    gamma_initializer=gamma_init,
    name=None,
):
    img_input = layers.Input(shape=input_img_size, name=name + "_img_input")
    x = img_input
    x = layers.Conv2D(filters, (7, 7), padding="same", kernel_initializer=kernel_init, use_bias=False)(x)
    x = keras.layers.GroupNormalization(groups=1, gamma_initializer=gamma_initializer)(x)
    x = layers.Activation("relu")(x)

    # Downsampling
    for _ in range(num_downsampling_blocks):
        filters *= 2
        x = downsample(x, filters=filters, activation=layers.Activation("relu"))

    # Residual blocks
    for _ in range(num_residual_blocks):
        x = residual_block(x, activation=layers.Activation("relu"))

    # Upsampling
    for _ in range(num_upsample_blocks):
        filters //= 2
        x = upsample(x, filters, activation=layers.Activation("relu"))

    # Final block
    x = layers.Conv2D(3, (7, 7), padding="same")(x)
    x = layers.Activation("tanh")(x)

    model = keras.models.Model(img_input, x, name=name)
    return model

## Discriminator (Function)

In [ ]:
def get_discriminator(filters=64, kernel_initializer=kernel_init, num_downsampling=3, name=None):
    img_input = layers.Input(shape=input_img_size, name=name + "_img_input")
    x = layers.Conv2D(filters,(4, 4),strides=(2, 2), padding="same", kernel_initializer=kernel_initializer,)(img_input)
    x = layers.LeakyReLU(0.2)(x)

    num_filters = filters
    for num_downsample_block in range(3):
        num_filters *= 2
        if num_downsample_block < 2:
            x = downsample(
                x,
                filters=num_filters,
                activation=layers.LeakyReLU(0.2),
                kernel_size=(4, 4),
                strides=(2, 2),
            )
        else:
            x = downsample(
                x,
                filters=num_filters,
                activation=layers.LeakyReLU(0.2),
                kernel_size=(4, 4),
                strides=(1, 1),
            )

    x = layers.Conv2D(1, (4, 4), strides=(1, 1), padding="same", kernel_initializer=kernel_initializer)(x)

    model = keras.models.Model(inputs=img_input, outputs=x, name=name)
    return model

## Creating Networks 

In [ ]:
#
#Get the generators
gen_G = get_resnet_generator(name="generator_G") # Translate features from monet to photos
gen_F = get_resnet_generator(name="generator_F") # reverse proces photos to monet

# Get the discriminators
disc_X = get_discriminator(name="discriminator_X") #Discrimination on monet real or fake
disc_Y = get_discriminator(name="discriminator_Y") #Discriminator on photo real or fake

# Cycle GAN 3. Teaching to paint class CycleGan

This is a little hard to follow but we basically need to update weights and unify all NN for them to work toghether

Some definition here:

**Lambda_cycle** = "cycle consistency loss" - generador G outputs : G(X) ---> gen F(G(x)) shoud go back to X, diference is an hyperparameter to tune

**Lambda_identity** = "identity loss" - Image from domain No. 2 on gen G shoud not make changes Y = G(Y).

In [ ]:
class CycleGan(keras.Model):
    def __init__(
        self,
        generator_G,
        generator_F,
        discriminator_X,
        discriminator_Y,
        lambda_cycle=10.0,
        lambda_identity=0.5,
    ):
        super().__init__() # calls init from parent class (keras.Model)
        self.gen_G = generator_G
        self.gen_F = generator_F
        self.disc_X = discriminator_X
        self.disc_Y = discriminator_Y
        self.lambda_cycle = lambda_cycle
        self.lambda_identity = lambda_identity
        
    def call(self, inputs):
        return (
            self.disc_X(inputs),
            self.disc_Y(inputs),
            self.gen_G(inputs),
            self.gen_F(inputs),
        )

    def compile(
        self,
        gen_G_optimizer,
        gen_F_optimizer,
        disc_X_optimizer,
        disc_Y_optimizer,
        gen_loss_fn,
        disc_loss_fn,
    ):
        super().compile()
        self.gen_G_optimizer = gen_G_optimizer
        self.gen_F_optimizer = gen_F_optimizer
        self.disc_X_optimizer = disc_X_optimizer
        self.disc_Y_optimizer = disc_Y_optimizer
        self.generator_loss_fn = gen_loss_fn
        self.discriminator_loss_fn = disc_loss_fn
        self.cycle_loss_fn = keras.losses.MeanAbsoluteError()
        self.identity_loss_fn = keras.losses.MeanAbsoluteError()

    def train_step(self, batch_data):
        # x is monet and y is photo
        real_x, real_y = batch_data

        # For CycleGAN, we need to calculate different
        # kinds of losses for the generators and discriminators.
        # We will perform the following steps here:
        #
        # 1. Pass real images through the generators and get the generated images
        # 2. Pass the generated images back to the generators to check if we
        #    can predict the original image from the generated image.
        # 3. Do an identity mapping of the real images using the generators.
        # 4. Pass the generated images in 1) to the corresponding discriminators.
        # 5. Calculate the generators total loss (adversarial + cycle + identity)
        # 6. Calculate the discriminators loss
        # 7. Update the weights of the generators
        # 8. Update the weights of the discriminators
        # 9. Return the losses in a dictionary

        with tf.GradientTape(persistent=True) as tape:
            # monet to fake photo
            fake_y = self.gen_G(real_x, training=True)
            # photo to fake monet -> y2x
            fake_x = self.gen_F(real_y, training=True)

            # Cycle (monet to fake photo to fake monet): x -> y -> x
            cycled_x = self.gen_F(fake_y, training=True)
            # Cycle (photo to fake moent to fake photo) y -> x -> y
            cycled_y = self.gen_G(fake_x, training=True)

            # Identity mapping
            same_x = self.gen_F(real_x, training=True)
            same_y = self.gen_G(real_y, training=True)

            # Discriminator output
            disc_real_x = self.disc_X(real_x, training=True)
            disc_fake_x = self.disc_X(fake_x, training=True)

            disc_real_y = self.disc_Y(real_y, training=True)
            disc_fake_y = self.disc_Y(fake_y, training=True)

            # Generator adversarial loss
            gen_G_loss = self.generator_loss_fn(disc_fake_y)
            gen_F_loss = self.generator_loss_fn(disc_fake_x)

            # Generator cycle loss
            cycle_loss_G = self.cycle_loss_fn(real_y, cycled_y) * self.lambda_cycle
            cycle_loss_F = self.cycle_loss_fn(real_x, cycled_x) * self.lambda_cycle

            # Generator identity loss
            id_loss_G = (
                self.identity_loss_fn(real_y, same_y)
                * self.lambda_cycle
                * self.lambda_identity
            )
            id_loss_F = (
                self.identity_loss_fn(real_x, same_x)
                * self.lambda_cycle
                * self.lambda_identity
            )

            # Total generator loss
            total_loss_G = gen_G_loss + cycle_loss_G + id_loss_G
            total_loss_F = gen_F_loss + cycle_loss_F + id_loss_F

            # Discriminator loss
            disc_X_loss = self.discriminator_loss_fn(disc_real_x, disc_fake_x)
            disc_Y_loss = self.discriminator_loss_fn(disc_real_y, disc_fake_y)

        # Get the gradients for the generators
        grads_G = tape.gradient(total_loss_G, self.gen_G.trainable_variables)
        grads_F = tape.gradient(total_loss_F, self.gen_F.trainable_variables)

        # Get the gradients for the discriminators
        disc_X_grads = tape.gradient(disc_X_loss, self.disc_X.trainable_variables)
        disc_Y_grads = tape.gradient(disc_Y_loss, self.disc_Y.trainable_variables)

        # Update the weights of the generators
        self.gen_G_optimizer.apply_gradients(
            zip(grads_G, self.gen_G.trainable_variables)
        )
        self.gen_F_optimizer.apply_gradients(
            zip(grads_F, self.gen_F.trainable_variables)
        )

        # Update the weights of the discriminators
        self.disc_X_optimizer.apply_gradients(
            zip(disc_X_grads, self.disc_X.trainable_variables)
        )
        self.disc_Y_optimizer.apply_gradients(
            zip(disc_Y_grads, self.disc_Y.trainable_variables)
        )

        return {
            "G_loss": total_loss_G,
            "F_loss": total_loss_F,
            "D_X_loss": disc_X_loss,
            "D_Y_loss": disc_Y_loss,
        }
        

## Loss Functions

In [ ]:
# Loss function for evaluating adversarial loss
adv_loss_fn = keras.losses.MeanSquaredError()

# Define the loss function for the generators
def generator_loss_fn(fake):
    fake_loss = adv_loss_fn(ops.ones_like(fake), fake)
    return fake_loss


# Define the loss function for the discriminators
def discriminator_loss_fn(real, fake):
    real_loss = adv_loss_fn(ops.ones_like(real), real)
    fake_loss = adv_loss_fn(ops.zeros_like(fake), fake)
    return (real_loss + fake_loss) * 0.5

## Checkpoint functions

In [ ]:
#Save images and checkpoints

class GANMonitor(keras.callbacks.Callback):
    """A callback to generate and show images after each epoch
    Modify 
     if (epoch + 1) % 1 == 0 for changing every X epoch
    Modify
     (self, num_img=4) for taking more pictures
    
    """

    def __init__(self, num_img=4):
        self.num_img = num_img

    def on_epoch_end(self, epoch, logs=None):
        
        if (epoch + 1) % 1 == 0: #Optionally recreate every x epoch
            
            _, ax = plt.subplots(4, 2, figsize=(12, 12))
            for i, img in enumerate(train_monet.take(self.num_img)):
                prediction = self.model.gen_G(img)[0].numpy() #AI paints
                prediction = (prediction * 127.5 + 127.5).astype(np.uint8) #Converts -1 to 1 to a readable image
                img = (img[0] * 127.5 + 127.5).numpy().astype(np.uint8)
    
                ax[i, 0].imshow(img)
                ax[i, 1].imshow(prediction)
                ax[i, 0].set_title("Input image")
                ax[i, 1].set_title("Translated image")
                ax[i, 0].axis("off")
                ax[i, 1].axis("off")

            plt.show()
            plt.close()
#End of class

painter = GANMonitor ()

"""
-------
Checkpoints
-------
"""
checkpoint_filepath = "/kaggle/working/model_checkpoints/cyclegan{epoch}_checkpoint.weights.h5"
checkpoint = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath, save_weights_only=True, save_freq=600
)

## Compile and train

In [ ]:
# Create cycle gan model
"""
---------------
Model from nothing
---------------
"""
cycle_gan_model = CycleGan(
    generator_G=gen_G, generator_F=gen_F, discriminator_X=disc_X, discriminator_Y=disc_Y
)

# Compile (final configuration) of the model

cycle_gan_model.compile(
    gen_G_optimizer=keras.optimizers.Adam(learning_rate=2e-4, beta_1=0.5),
    gen_F_optimizer=keras.optimizers.Adam(learning_rate=2e-4, beta_1=0.5),
    disc_X_optimizer=keras.optimizers.Adam(learning_rate=2e-4, beta_1=0.5),
    disc_Y_optimizer=keras.optimizers.Adam(learning_rate=2e-4, beta_1=0.5),
    gen_loss_fn=generator_loss_fn,
    disc_loss_fn=discriminator_loss_fn,
)

"""
----------------
Model from checkpoint - if want to retrain after x epoch
---------------
"""
# cycle_gan_model.load_weights("/kaggle/working/model_checkpoints/cyclegan_weights_epoch05.h5")

In [ ]:
#debugging
mini_monet = train_monet.take(5)
mini_photos = train_photos.take(5)

In [ ]:
# Here we will train the model 
cycle_gan_model.fit(tf.data.Dataset.zip((train_monet, train_photos)),
    epochs=100,
    callbacks=[painter, checkpoint]
)

# Results - After the training

So with that preview i have to say it was very good for making Fake Monet from real ones. Is time to see if it can also paint a landscape.
I have to say that learning is no allways consistent, and some epochs are better than others. GAN training is unstable and has some oscillations. Also some pictures seems "easier" than others wich can mean filters are better for capturing their essence on ResNet. 

Delicate balance between the generator and discriminator— is hard to mantein and if one starts to overpower the other, the quality can drop.

Another explanations on inestability are because os:

**Mode Collapse:** The generator might start to produce a limited variety of outputs (mode collapse) after a certain point, leading to less diverse or lower-quality images.

**Noisy Training Signal:** The adversarial loss can be quite noisy, causing fluctuations in image quality as the model continually adjusts to the discriminator's feedback.

## Final appreciations

The main and big difference from original to fake can be the little details, so i went asking to a famous transformer (CHAT-GPT) what can be the reasons and this is "his" answer:
"
Downsampling/Upsampling Loss:
    Many generators include downsampling (to capture global context) followed by upsampling. This process can cause a loss of fine details because high-frequency information may be smoothed out during downsampling.

Kernel Size and Receptive Field:
    Convolutional layers have a limited receptive field determined by the kernel size. If the kernels are too large or the network isn’t deep enough, they might not focus on fine details, leading to a blurred reconstruction of high-frequency signals.

Loss Function Emphasis:
    Standard adversarial and cycle consistency losses often focus on global structure rather than fine textures. Without an additional loss (like perceptual or texture loss), the generator might prioritize overall coherence at the expense of sharp, detailed features.

Architecture Choices:
    Certain architectural choices (e.g., the use of transposed convolutions) can introduce artifacts or fail to perfectly reconstruct high-frequency components.

In practice, if preserving high-frequency details is important, researchers often experiment with different kernel sizes, skip connections, or add extra loss terms that specifically encourage the retention of fine details.

"

I guess we need to play with a little bit more with Learning Rate and Hyperparameters, maybe even padding or more advancer preproccessing if you want better and more consistant images. But this has already taken a lot of time. And i am preatty happy with what we did and leaned here.
    

In [ ]:
"""
----------------
Model from checkpoint - Saving for using it later on

Use favorite epoc weights and don't .fit an you will have Fake Monet from previous epoch

28, 31, 35, 45, 53, 55, and 71 are promising
After loading wights you could safe hole model
tf.saved_model.save(cycle_gan_model, "/kaggle/working/saved_cyclegan_model") 
---------------

cycle_gan_model.load_weights("/kaggle/working/model_checkpoints/cyclegan71_checkpoint.weights.h5")
"""


In [ ]:
# Show painted the landscapes
_, ax = plt.subplots(4, 2, figsize=(10, 15))
for i, img in enumerate(train_photos.take(4)):
    prediction = cycle_gan_model.gen_F(img, training=False)[0].numpy()
    prediction = (prediction * 127.5 + 127.5).astype(np.uint8)
    img = (img[0] * 127.5 + 127.5).numpy().astype(np.uint8)

    ax[i, 0].imshow(img)
    ax[i, 1].imshow(prediction)
    ax[i, 0].set_title("Input image")
    ax[i, 1].set_title("Translated image")
    ax[i, 0].axis("off")
    ax[i, 1].axis("off")

plt.tight_layout()
plt.show()

In [ ]:
# rm -rf /kaggle/working/saved_cyclegan